In [ ]:
import re
import itertools
from google.colab import files

#CFG Rules File
uploaded = files.upload()
input_path = next(iter(uploaded.keys()))

# Load CFG Rules
def load_cfg_from_file(path):
    rules = []
    with open(path, 'r') as f:
        rule_lines = [line.strip().strip(',') for line in f if line.strip()]
    rules = []
    for line in rule_lines:
        match = re.match(r"\('([^']+)',\s*\[([^\]]+)\]\)", line)
        if match:
            lhs = match.group(1)
            rhs = [sym.strip().strip("'") for sym in match.group(2).split(',')]
            rules.append((lhs, rhs))
    return rules

In [ ]:
#CNF Converter
class CNFConverter:
    def __init__(self, rules):
        self.original_rules = rules
        self.new_rules = []
        self.term_map = {}
        self.counter = itertools.count(1)

    def _get_new_nonterminal(self, base='X'):
        return f'{base}{next(self.counter)}'

    def convert(self):
        rules = self.original_rules[:]
        updated_rules = []

        for lhs, rhs in rules:
            new_rhs = []
            for sym in rhs:
                if len(rhs) > 1 and not re.fullmatch(r"[A-Z$]+", sym):
                    if sym not in self.term_map:
                        nt = self._get_new_nonterminal('T')
                        self.term_map[sym] = nt
                        self.new_rules.append((nt, [sym]))
                    new_rhs.append(self.term_map[sym])
                else:
                    new_rhs.append(sym)
            updated_rules.append((lhs, new_rhs))

        for lhs, rhs in updated_rules:
            while len(rhs) > 2:
                first, second, *rest = rhs
                new_nt = self._get_new_nonterminal()
                self.new_rules.append((lhs, [first, new_nt]))
                lhs = new_nt
                rhs = [second] + rest
            self.new_rules.append((lhs, rhs))

        return self.new_rules

def write_cnf_to_file(rules, path):
    with open(path, 'w') as f:
        for lhs, rhs in rules:
            f.write(f"('{lhs}', {rhs}),\n")

cfg_rules = load_cfg_from_file(input_path)
converter = CNFConverter(cfg_rules)
cnf_rules = converter.convert()
output_path = 'cnf_rules.txt'
write_cnf_to_file(cnf_rules, output_path)
files.download(output_path)

In [ ]:
#CYK parser

def cyk_parse(words, grammar, start_symbol='S'):
    """
    :param words: list of words in the input sentence
    :param grammar: list of (lhs, rhs) rules in CNF, e.g. ('S', ['NP', 'VP'])
    :param start_symbol: the start symbol of the grammar
    :return: (table, back) where table[l][s][X] = True iff X derives words[s:s+l]
    """
    n = len(words)
    # Extract all non-terminals
    nonterminals = sorted({lhs for lhs, rhs in grammar})
    nt_index = {nt: i for i, nt in enumerate(nonterminals)}

    # Table for P[l][s][v] as in pseudocode (using a dict for efficiency)
    table = [[dict() for _ in range(n)] for _ in range(n+1)]
    back = [[dict() for _ in range(n)] for _ in range(n+1)]

    # Preprocess rules
    unary_rules = {}  # a -> [A]
    binary_rules = [] # (A, B, C)

    for lhs, rhs in grammar:
        if len(rhs) == 1:
            unary_rules.setdefault(rhs[0], []).append(lhs)
        elif len(rhs) == 2:
            binary_rules.append((lhs, rhs[0], rhs[1]))

    # Initialize for length 1 substrings
    for s in range(n):
        for A in unary_rules.get(words[s], []):
            table[1][s][A] = True
            back[1][s][A] = (words[s],)

    # Main CYK loop
    for l in range(2, n+1):  # Length of span
        for s in range(n-l+1):  # Start of span
            for p in range(1, l):  # Partition
                left_cell = table[p][s]
                right_cell = table[l-p][s+p]
                for A, B, C in binary_rules:
                    if B in left_cell and C in right_cell:
                        table[l][s][A] = True
                        back[ l ][ s ][ A ] = (p, B, C)

    # Is the input in the language?
    accepted = start_symbol in table[n][0]

    return accepted, table, back, nonterminals

def build_tree(back, l, s, symbol):
    # Recursively reconstruct the parse tree
    entry = back[l][s][symbol]
    if isinstance(entry, tuple) and len(entry) == 1:
        return (symbol, entry[0])  # Terminal
    else:
        p, B, C = entry
        left = build_tree(back, p, s, B)
        right = build_tree(back, l-p, s+p, C)
        return (symbol, left, right)

from nltk import Tree


In [ ]:
# Simple Example

# Load grammar from cnf_rules.txt
def load_cnf_rules(filepath):
    rules = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip().strip(',')
            if line:
                try:
                    lhs, rhs_str = line.split(', [')
                    lhs = lhs.strip("('").strip("'")
                    rhs = [sym.strip().strip("'") for sym in rhs_str.strip("])").split(',')]
                    rules.append((lhs, rhs))
                except ValueError:
                    print(f"Skipping invalid line: {line}")
    return rules

# Load the grammar and then append the rule
grammar = load_cnf_rules('cnf_rules (10).txt')
grammar.append(('CC', ['CC']))
grammar.append(('IN', ['IN']))
grammar.append(('VBG', ['VBG']))
grammar.append(('CD', ['CD']))   # Cardinal number
grammar.append(('DT', ['DT']))   # Determiner
grammar.append(('EX', ['EX']))   # Existential there
grammar.append(('FW', ['FW']))   # Foreign word
grammar.append(('JJ', ['JJ']))   # Adjective
grammar.append(('JJR', ['JJR'])) # Adjective, comparative
grammar.append(('JJS', ['JJS'])) # Adjective, superlative
grammar.append(('LS', ['LS']))   # List item marker
grammar.append(('MD', ['MD']))   # Modal
grammar.append(('NN', ['NN']))   # Noun, singular or mass
grammar.append(('NNS', ['NNS'])) # Noun, plural
grammar.append(('NNP', ['NNP'])) # Proper noun, singular
grammar.append(('NNPS', ['NNPS'])) # Proper noun, plural
grammar.append(('PDT', ['PDT'])) # Predeterminer
grammar.append(('POS', ['POS'])) # Possessive ending
grammar.append(('PRP', ['PRP'])) # Personal pronoun
grammar.append(('PP', ['PP']))   # Possessive pronoun
grammar.append(('RB', ['RB']))   # Adverb
grammar.append(('RBR', ['RBR'])) # Adverb, comparative
grammar.append(('RBS', ['RBS'])) # Adverb, superlative
grammar.append(('RP', ['RP']))   # Particle
grammar.append(('SYM', ['SYM'])) # Symbol
grammar.append(('TO', ['TO']))   # to
grammar.append(('UH', ['UH']))   # Interjection
grammar.append(('VB', ['VB']))   # Verb, base form
grammar.append(('VBD', ['VBD'])) # Verb, past tense
grammar.append(('VBN', ['VBN'])) # Verb, past participle
grammar.append(('VBP', ['VBP'])) # Verb, non-3rd person singular present
grammar.append(('VBZ', ['VBZ'])) # Verb, 3rd person singular present
grammar.append(('WDT', ['WDT'])) # wh-determiner
grammar.append(('WP', ['WP']))   # wh-pronoun
grammar.append(('WP$', ['WP$'])) # Possessive wh-pronoun
grammar.append(('WRB', ['WRB'])) # wh-adverb
grammar.append(('S', ['NP', 'VP']))
grammar.append(('NP', ['DT', 'NN']))
grammar.append(('VP', ['VBZ', 'NP']))
grammar.append(('VP', ['VBZ', 'JJ']))
grammar.append(('PP', ['IN', 'NP']))
grammar.append(('NP', ['NN']))
grammar.append(('NP', ['PRP']))
grammar.append(('VP', ['VBD', 'PP']))
grammar.append(('VP', ['VB', 'ADJP']))
grammar.append(('ADJP', ['JJ']))
grammar.append(('PRP$', ['PRP$']))



In [ ]:
!pip install -q spacy


In [ ]:
# POS Tagger
import sys
import subprocess
import spacy

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Ensure English model is installed
try:
    spacy.load("en_core_web_sm")
except OSError:
    print("English model not found. Downloading...")
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])

# Load the English model
nlp = spacy.load("en_core_web_sm")


# Universal POS tags to CFG map (adaptable for different languages)
cfg_map = {
    "NOUN": "N",
    "PROPN": "N",
    "PRON": "Pro",
    "VERB": "V",
    "AUX": "V",
    "ADJ": "Adj",
    "ADV": "Adv",
    "DET": "D",
    "ADP": "P",
    "CCONJ": "Conj",
    "SCONJ": "Conj",
    "PART": "Part",
    "INTJ": "Intj",
    "NUM": "Num",
    "PUNCT": ".",
}

text = input("Enter a sentence: ")

print(f"\nProcessing in English:")

doc = nlp(text)
sentence = []
print("\nCFG-style POS Tags:\n")
for token in doc:
    tag = cfg_map.get(token.tag_, token.tag_)
    sentence.append(tag)

print(sentence)

Enter a sentence: the john eats he

Processing in English:

CFG-style POS Tags:

the             → DT
john            → NNP
eats            → VBZ
he              → PRP
['DT', 'NNP', 'VBZ', 'PRP']


In [ ]:
# Test Run
accepted, table, back, nts = cyk_parse(sentence, grammar, start_symbol='S')
if accepted:
    print("Grammatically Correct!")
    tree = build_tree(back, len(sentence), 0, 'S')
    print("Parse tree:", tree)
else:
    print("Grammatically Incorrect!")
for l in range(1, len(sentence)+1):
    for s in range(len(sentence)-l+1):
        print(f"Span length {l}, start {s}: {table[l][s]}")



Grammatically Correct!
Parse tree: ('S', ('NP', ('DT', 'DT'), ('NNP', 'NNP')), ('VP', ('VBZ', 'VBZ'), ('NP', 'PRP')))
Span length 1, start 0: {'NP': True, 'DT': True}
Span length 1, start 1: {'NP': True, 'NNP': True}
Span length 1, start 2: {'VP': True, 'VBZ': True}
Span length 1, start 3: {'NP': True, 'PRP': True}
Span length 2, start 0: {'NP': True}
Span length 2, start 1: {'S': True, 'NP': True, 'X10': True, 'X47': True, 'X54': True}
Span length 2, start 2: {'VP': True}
Span length 3, start 0: {'NP': True, 'X18': True, 'X53': True, 'X58': True, 'S': True, 'X10': True, 'X47': True, 'X54': True}
Span length 3, start 1: {'S': True, 'NP': True, 'X10': True, 'X47': True, 'X54': True}
Span length 4, start 0: {'NP': True, 'X18': True, 'X53': True, 'X58': True, 'S': True, 'X10': True, 'X47': True, 'X54': True}
